In [3]:
from numpy import array, sin, cos
from numpy.linalg import norm
from lib.convert import deg2rad, rad2deg
from lib.rotation import Rx, Ry, Rz
from lib.geodesy import geod2ECEF, ECEF2geod

# System conditions
dt = 1                          # second
R = 6371e3                      # meter

oie_e = array([[0],             
               [0],
               [7292115e-11]])  # radian/second (Earth rotation)

geb_g = array([[0],             
               [0],
               [9.795]])        # meter/second^2 (Gravity at altitude h)      


# Initial conditions
lat = deg2rad(45)               # radian
lon = -deg2rad(30)              # radian
h = 10000                       # meter

# Compute position i e-frame
peb_e = geod2ECEF(R, R, lat, lon, h)

yaw = deg2rad(60)               # radian
pitch = 0                       # radian
roll  = 0                       # radian

veb_b = array([[800000/3600],
               [0],
               [0]])

# Observations
# Gyroscope
gyrox = -deg2rad(0.1)           # radian/second
gyroy = deg2rad(0.1)            # radian/second
gyroz = -deg2rad(1.0)           # radian/second

oib_b = array([[gyrox],         # radian/second
               [gyroy],
               [gyroz]])

# Accelerometer
accx = 0                        # meter/second^2
accy = -3.9                     # meter/second^2
accz = -10.1                    # meter/second^2

fib_b = array([[accx],          # meter/second^2
               [accy],
               [accz]])

# Local functions
# Create cross product matrix
def crossm(a):
    
    A = array([[ 0, -a[2, 0], a[1, 0]],
               [ a[2, 0], 0, -a[0, 0]],
               [-a[1, 0], a[0, 0], 0]])
    
    return A

# Normalize rotation matrix
def normalize(C):
    return (3/2)*C - (1/2)*C@C.T@C

# Compute Cb_g
Cb_g = Rz(yaw)@Ry(pitch)@Rx(roll)

# Compute Ce_g
Ce_g = array([[-sin(lat)*cos(lon), -sin(lat)*sin(lon), cos(lat)],
              [-sin(lon), cos(lon), 0],
              [-cos(lat)*cos(lon), -cos(lat)*sin(lon), -sin(lat)]])

# Compute Cb_e
Cb_e = Ce_g.T@Cb_g

# Update Cb_e
oeb_b = oib_b - Cb_e.T@oie_e
Oeb_b = crossm(oeb_b)
Cb_e_dot = Cb_e@Oeb_b

# Estimate Cb_e
Cb_e_hat = Cb_e + Cb_e_dot*dt

# Normalize (Cb_e)
Cb_e_hat = normalize(Cb_e_hat)

# Compute Coriolis component
Oie_e = crossm(oie_e)

# Compute velocity in e-frame
veb_e = Cb_e_hat@veb_b

# Compute acceleration in e-frame
aeb_e = Cb_e_hat@fib_b - 2*Oie_e@veb_e + Ce_g.T@geb_g

# Compute position in e-frame
peb_e = peb_e + veb_e*dt + 1/2*aeb_e*dt**2

# Compute position in e-frame
llh = ECEF2geod(R, R, peb_e)

lat = llh[0]; lon = llh[1]; h = llh[2]

print(rad2deg(lat))
print(rad2deg(lon))
print(h)

45.000952100770355
-30.00247992269818
10000.51729558874
